In [ ]:
import pims
import numpy as np
import xarray
import warnings
import nd2
import dask
import dask_image.imread

In [ ]:
file_nd2 = "../tests/data/testData10frames.nd2"
# file_nd2 = "../data/20210503_18_a_.nd2"
file_tif = "../tests/data/21-03-31_ddm_water_control_sample.tif"
# file_tif = "../data/sim_diffusion.tif"
file_lif = "../tests/data/testData1series.lif"

### Pims reader

In [ ]:
def get_reader(path: str):
    with open(path, "rb"):
        return pims.Bioformats(path)

In [ ]:
rdr = get_reader(file_nd2)

In [ ]:
data = np.asarray(rdr.get_frame(0))
type(data)

### Import delayed image stacks

In [ ]:
warnings.filterwarnings("ignore", message="Due to an issue with JPype 0.6.0")
img_nd2 = dask_image.imread.imread(file_nd2, nframes=10)
img_nd2.shape

In [ ]:
img_tif = dask_image.imread.imread(file_tif, nframes=10)
img_tif

In [ ]:
img_lif = dask_image.imread.imread(file_lif, nframes=10)
img_lif

### Read metadata

In [ ]:
def read_metadata_nd2(filename):
    with nd2.ND2File(filename) as imgs:
        tscale = imgs.experiment[0].parameters.periodDiff.avg
        xscale = imgs.voxel_size().x
    return xscale, tscale

def read_metadata_lif(filename):
    with pims.Bioformats(filename) as imgs:
        xscale = imgs.metadata.PixelsPhysicalSizeX(0)
        tscale = imgs.metadata.PlaneDeltaT(0, 1) * 1000.0
    return xscale, tscale

def read_metadata_tif(filename):
    with pims.Bioformats(filename) as imgs:
        try:
            xscale = imgs.metadata.PixelsPhysicalSizeX(0)            
        except AttributeError:
            xscale = 1
        try: 
            tscale = imgs.metadata.PixelsTimeIncrement(0) * 1000.0
        except AttributeError:
            tscale = 1
        
    return xscale, tscale

In [ ]:
read_metadata_lif(file_lif)

In [ ]:
read_metadata_nd2(file_nd2)

In [ ]:
read_metadata_tif(file_tif)

### Create xarray 

In [ ]:
def create_xarray(arr: dask.array.core.Array, xscale: float, tscale: float) -> xarray.DataArray:
    # Create coordinates
    t_coords = np.arange(0.0, arr.shape[0] * tscale, tscale)
    x_coords = np.arange(0.0, arr.shape[1] * xscale, xscale)
    y_coords = np.arange(0.0, arr.shape[2] * xscale, xscale)
    x_arr = xarray.DataArray(
        data=arr,
        dims=["T", "Y", "X"],
        coords=(t_coords, y_coords, x_coords),
        attrs=dict(xyScale=xscale, tScale=tscale),
    )
    return x_arr

In [ ]:
img_nd2 = dask_image.imread.imread(file_nd2, nframes=10)
xscale, tscale = read_metadata_nd2(file_nd2)
out = create_xarray(img_nd2, xscale, tscale)
out


### Create single function call

In [ ]:
def import_data(filename, nframes=5):
    d_arr = dask_image.imread.imread(filename, nframes)
    xscale, tscale = read_metadata_nd2(file_nd2)
    return create_xarray(d_arr, xscale, tscale)


In [ ]:
x_arr = import_data(file_nd2)
x_arr

In [ ]:
x_arr = import_data(file_lif)
x_arr

In [ ]:
x_arr = import_data(file_tif)
x_arr

### Lif multiple experiments support

In [ ]:
import pims
import numpy as np
filename = "../tests/data/testData3series.lif"
experiment = 3
with pims.open(filename, series=experiment) as imgs:
    # imgs.series = experiment
    arr = np.asarray(imgs)
arr.shape

### Use refactored functions

In [ ]:
import warnings
from ddm.data_handling.read_metadata import read_metadata
warnings.filterwarnings("ignore", message="Due to an issue with JPype 0.6.0")

filename = "../tests/data/testData1series.lif"

metadata = read_metadata(filename)

# xscale = None
# metadata["xscale"] = metadata["xscale"] if xscale is None else xscale
type(metadata["tscale"])

In [ ]:
from ddm.data_handling.read_file import select_experiment
out = select_experiment(metadata, experiment=0)
out

In [ ]:
from ddm.data_handling.dask_image import read_data_into_dask
filename = "../tests/data/testData3series.lif"
out = read_data_into_dask(filename, nframes=2, experiment=5)
out

In [ ]:
from ddm.data_handling.read_file import load_data
filename = "../tests/data/testData1series.lif"
out = load_data(filename, 5, experiment=1)
out

In [ ]:
from ddm.data_handling.dask_image import _read_frame
filename = "../tests/data/testData3series.lif"
arr = _read_frame(filename, i=slice(0,5), experiment=4)
# arr.shape


In [ ]:
from ddm.data_handling.read_file import read_file
filename = "../tests/data/testData1series.lif"
out = read_file(filename, nframes=5)
out

In [ ]:
from ddm.data_handling.read_file import read_file
filename = "../data/sim_diffusion.tif"
out = read_file(filename, nframes=5)
out

In [ ]:
from ddm.data_handling.read_file import read_file
filename = "../data/A1_s3001.nd2"
out = read_file(filename, nframes=5)
out

In [ ]:
import pims
import numpy as np

filename = "../tests/data/testData1series.lif"
sfname = str(filename)
with pims.Bioformats(sfname, series=0) as imgs:
    shape = (len(imgs),) + imgs.frame_shape
    dtype = np.dtype(imgs.pixel_type)

shape